In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook was authored with assistance from [Ivan Nardini](https://github.com/inardini)

# Get started with Vertex AI Experiments

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fget_started_with_vertex_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview


This tutorial covers getting started with Vertex AI Experiments, and demonstrates how to use Vertex AI in production.

Learn more about [Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments), [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata) and [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training).

## Get started

### Install Vertex AI SDK for Python and other required packages


In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [5]:
BUCKET_URI = "gs://cours1bucket" # f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
! gsutil mb -l $LOCATION $BUCKET_URI

Creating gs://cours1bucket/...
ServiceException: 409 A Cloud Storage bucket named 'cours1bucket' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [7]:
SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" # "[your-service-account]"  # @param {type:"string"}

In [8]:
IS_COLAB = False

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        # print("shell_output=", shell_output)
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries

In [9]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [10]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Introduction to Vertex AI Experiments

With Vertex AI Experiments you can log and track the following when experimenting and developing your model architecture and model training:

- Log the metaparameters for the model architecture.
- Log the hyperparameters for training.
- Log the evaluation metrics.
- Create an artifact lineage of the dataset, model and evaluation.
- Group one or more training runs under an experiment.
- Compare experiments.

Vertex AI Experiments can be integrated with the following development process flows:

- Local development in a notebook
- Cloud development in Vertex AI Training
- Operationalizing development in Vertex AI Pipelines

Learn more about [Experiments]( https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).

Learn more about [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

In [24]:
# Specify a name for the experiment
EXPERIMENT_NAME = "supaero-vertex-course-1-1"

if EXPERIMENT_NAME == "[your-experiment-name]":
    EXPERIMENT_NAME = f"example-{uuid.uuid1()}"

In [25]:
import random

units_choice = [32, 64, 128, 256]
epochs_choice = [1, 10, 100, 1000]
batch_size_choice = [1, 8, 32, 64]
learning_rate_choice = [.0001, .01, .1, 1]

# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)

for i in range(1, 11):
    aiplatform.start_run(f"run-{str(i)}")

    metaparams = {}
    metaparams["units"] = random.choice(units_choice)
    aiplatform.log_params(metaparams)

    hyperparams = {}
    hyperparams["epochs"] = random.choice(epochs_choice)
    hyperparams["batch_size"] = random.choice(batch_size_choice)
    hyperparams["learning_rate"] = random.choice(learning_rate_choice)
    aiplatform.log_params(hyperparams)

    metrics = {}
    metrics["test_acc"] = random.random() * 30 + 68
    metrics["train_acc"] = min(random.random() * 3 + metrics["test_acc"], 100)

    aiplatform.log_metrics(metrics)
    
    aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-1 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-2 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-3 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-4 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-5 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-6 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-7 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-8 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-9 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-10 to Experiment: supaero-vertex-course-1-1


In [21]:
experiment_df

,experiment_name,run_name,run_type,state,param.batch_size,param.learning_rate,param.epochs,param.units,metric.train_acc,metric.test_acc
0,supaero-course-1-1,run-10,system.ExperimentRun,COMPLETE,8.0,0.0001,100.0,256.0,100.000000,95.712354
1,supaero-course-1-1,run-9,system.ExperimentRun,COMPLETE,64.0,0.1000,10.0,128.0,101.117122,99.558977
2,supaero-course-1-1,run-8,system.ExperimentRun,COMPLETE,64.0,0.0100,1.0,64.0,100.000000,84.759412
3,supaero-course-1-1,run-7,system.ExperimentRun,COMPLETE,1.0,1.0000,1.0,64.0,100.000000,84.887640
4,supaero-course-1-1,run-6,system.ExperimentRun,COMPLETE,64.0,0.0100,100.0,128.0,100.000000,91.225823
5,supaero-course-1-1,run-5,system.ExperimentRun,COMPLETE,64.0,1.0000,1.0,32.0,102.270010,99.515128
6,supaero-course-1-1,run-4,system.ExperimentRun,COMPLETE,1.0,0.1000,100.0,128.0,100.000000,89.742437
7,supaero-course-1-1,run-3,system.ExperimentRun,COMPLETE,8.0,1.0000,100.0,32.0,100.000000,95.709242
8,supaero-course-1-1,run-2,system.ExperimentRun,COMPLETE,64.0,0.1000,10.0,128.0,100.000000,91.013360
9,supaero-course-1-1,run-1,system.ExperimentRun,COMPLETE,32.0,0.0100,100.0,128.0,100.000000,93.166421


In [51]:
metrics = {}
metrics["test_acc"] = 98.7
metrics["train_acc"] = 99.3
aiplatform.log_metrics(metrics)

#### Get the experiment results

When you are finished with a run within an experiment, call `end_run()` method to complete the logging for that run.

Next, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In [54]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                            0                         1
experiment_name      experiment-test-course-1  experiment-test-course-1
run_name                                run-2                     run-1
run_type                 system.ExperimentRun      system.ExperimentRun
state                                COMPLETE                  COMPLETE
param.batch_size                         32.0                      32.0
param.epochs                            100.0                     100.0
param.units                             256.0                     128.0
param.learning_rate                      0.01                      0.01
metric.test_acc                          98.8                      98.7
metric.train_acc                         99.5                      99.3


#### Delete the experiment

Next, delete the experiment using the `delete()` method.

In [55]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)

Experiment run run-2 skipped backing tensorboard run deletion.
To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"experiment-test-course-1-run-2-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()


To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"experiment-test-course-1-run-2-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()


Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2


INFO:google.cloud.aiplatform.base:Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2


Context deleted. . Resource name: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2


...

INFO:google.cloud.aiplatform.base:Deleting Context resource: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1


Delete Context backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1/operations/5446936520457977856


INFO:google.cloud.aiplatform.base:Delete Context backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1/operations/5446936520457977856


Context resource projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1 deleted.


INFO:google.cloud.aiplatform.base:Context resource projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1 deleted.
